## Mini-Project #3

In this mini-project, we will answer the following questions:
- Are you more or less likely to be injured in a bike-related accident if you are on a bike path or not? How would the results differ for injuries of different severity, zip codes, and by the type of other party in the accident?
- In which parts of the city are you more likely to get into an alcohol-related accident? How can you explain that using additional statistics by zip codes, for example, by the number of alcohol-serving bars within the zip code? 

These questions will be answered using Arcgis API for Python.

### If you want to skip to see the [Results and Conclusion](#conclusion-bullet)

### Part 1 (5 points)

Recall the very first demo during the first lecture: we searched available content for "bike san diego", and found several layers or services, including collisions and bike routes. The collisions file showed bike and pedestrian TIMS (Transpotation Injury Mapping System) geocoded collisions in SD County. You can view the system description, including definitions of fields, at https://tims.berkeley.edu/help/SWITRS.php (SWITRS == Statewide Integrated Traffic Records System from CA Highway Patrol). There is a field indicating accidents that involved bicyclists. 

To answer the questions noted above, we'll need to integrate bike paths and accidents. However, notice that the data we found through ArcGIS during the first lecture were for different times. Therefore, our first task is to grab more recent data and load it into this notebook.

To do so, please follow these steps:
1. Create an account at https://tims.berkeley.edu/, and sign in.
1. At SWITRS Query and Map (under Tools), make a request for data from 1/1/2014 to 12/31/2018, for San Diego city.  Specify that you are interested in bicycle collisions (under Collision factors), and run the query.
1. Download the collision data as a csv file. Also, download party and victims data, because some attributes you'd need are in these files.
1. Explore the data you downloaded, and understand the fields you need to work with. See SWITRS FAQ page at https://tims.berkeley.edu/help/Query_and_Map.php#FAQs and the codebook at https://tims.berkeley.edu/help/SWITRS.php#Codebook. SWITRS is relatively well documented.
1. Find the field that is common for the 3 tables (this is what you can use to join them). Note, also, that there are similarly-named fields in the three tables, but they apply to different types. For example, "party sobriety" is a field in the "party" table (here, you can tell who in the collision was under the influence), while there is also an "alchohol involved" field in the collision table.

In [202]:
# Package imports go here.

%matplotlib inline 
import geopandas as gpd
from shapely.geometry import Point
import folium
from arcgis import *
import pandas as pd
pd.options.mode.chained_assignment = None
from IPython.display import display
import seaborn as sn

gis = GIS(username= 'rherlim_dsc170fa20') #Bdaman2000

Enter password: ········


In [203]:
# Load the data (using content manager, and also adding downloaded SWITRS datasets), retain only fields we need

#This is the feature layer for 2018 bike routes
#We chose 2018 because that's the most recent year available for bike collisions on TIMS

sd_bike_routes = gis.content.get('31ef0a6608e04d7e9255e48b10658424')
bike_routes_fl = sd_bike_routes.layers[0]
# for field in bike_routes_fl.properties.fields:
#     print(field['name'])
bike_routes_fs = bike_routes_fl.query()
bike_routes_sdf = bike_routes_fs.sdf
print(bike_routes_sdf.columns)
bike_routes_sdf.head()

Index(['FID', 'RD20FULL', 'Jurisdicti', 'ROUTE', 'Route_Clas', 'Max_Elev',
       'Shape_Leng', 'Shape__Length', 'SHAPE'],
      dtype='object')


,FID,RD20FULL,Jurisdicti,ROUTE,Route_Clas,Max_Elev,Shape_Leng,Shape__Length,SHAPE
0,1,AVOCADO AV,El Cajon,2,Bike Lane,733.589062,28.294438,28.294349,"{""paths"": [[[6343793.60395551, 1859750.1010223..."
1,2,BALTIMORE DR,San Diego,2,Bike Lane,596.089307,869.178045,869.178242,"{""paths"": [[[6320428.64412384, 1866900.3731390..."
2,3,BIKEWAY,San Diego,1,Multi-Use Path,589.489010,3684.416275,3684.416375,"{""paths"": [[[6296431.99986, 1917146.00000983],..."
3,4,03RD AV,San Diego,8,Other Suggested Routes,297.958873,352.068928,352.069043,"{""paths"": [[[6281197.00014634, 1854575.0001521..."
4,5,ALVARADO ST,Oceanside,3,Bike Route,49.859613,338.540320,338.540404,"{""paths"": [[[6220764.99896334, 2010895.9998454..."


In [204]:
# Load csv files

# Your code goes here
collisions = pd.read_csv('Collisions.csv')
parties = pd.read_csv('Parties.csv')
victims = pd.read_csv('Victims.csv')

display(collisions.head())
print("Collisions shape: " + str(collisions.shape))
print(collisions.columns)

display(parties.head())
print("Parties shape: " + str(parties.shape))
print(parties.columns)

display(victims.head())
print("Victims shape: " + str(victims.shape))
print(victims.columns)

,CASE_ID,ACCIDENT_YEAR,PROC_DATE,JURIS,COLLISION_DATE,COLLISION_TIME,OFFICER_ID,REPORTING_DISTRICT,DAY_OF_WEEK,CHP_SHIFT,...,COUNT_MC_KILLED,COUNT_MC_INJURED,PRIMARY_RAMP,SECONDARY_RAMP,LATITUDE,LONGITUDE,COUNTY,CITY,POINT_X,POINT_Y
0,6291500,2014,2014-10-29,3711,2014-08-06,1624,4653,NaN,3,5,...,0,0,-,-,NaN,NaN,SAN DIEGO,SAN DIEGO,-117.103594,32.911904
1,6291600,2014,2016-01-12,3711,2014-10-07,2312,5051,NaN,2,5,...,0,0,-,-,NaN,NaN,SAN DIEGO,SAN DIEGO,-117.205080,32.754250
2,6291825,2014,2015-01-09,3711,2014-11-09,1729,6366,NaN,7,5,...,0,0,-,-,NaN,NaN,SAN DIEGO,SAN DIEGO,-117.136246,32.750301
3,6328108,2014,2014-06-02,3711,2014-01-04,1849,4495,NaN,6,5,...,0,0,-,-,NaN,NaN,SAN DIEGO,SAN DIEGO,-117.146743,32.789146
4,6337228,2014,2014-06-05,3711,2014-01-10,1754,6430,SANDI,5,5,...,0,0,-,-,NaN,NaN,SAN DIEGO,SAN DIEGO,-117.233150,32.833450


Collisions shape: (2291, 80)
Index(['CASE_ID', 'ACCIDENT_YEAR', 'PROC_DATE', 'JURIS', 'COLLISION_DATE',
       'COLLISION_TIME', 'OFFICER_ID', 'REPORTING_DISTRICT', 'DAY_OF_WEEK',
       'CHP_SHIFT', 'POPULATION', 'CNTY_CITY_LOC', 'SPECIAL_COND', 'BEAT_TYPE',
       'CHP_BEAT_TYPE', 'CITY_DIVISION_LAPD', 'CHP_BEAT_CLASS', 'BEAT_NUMBER',
       'PRIMARY_RD', 'SECONDARY_RD', 'DISTANCE', 'DIRECTION', 'INTERSECTION',
       'WEATHER_1', 'WEATHER_2', 'STATE_HWY_IND', 'CALTRANS_COUNTY',
       'CALTRANS_DISTRICT', 'STATE_ROUTE', 'ROUTE_SUFFIX', 'POSTMILE_PREFIX',
       'POSTMILE', 'LOCATION_TYPE', 'RAMP_INTERSECTION', 'SIDE_OF_HWY',
       'TOW_AWAY', 'COLLISION_SEVERITY', 'NUMBER_KILLED', 'NUMBER_INJURED',
       'PARTY_COUNT', 'PRIMARY_COLL_FACTOR', 'PCF_CODE_OF_VIOL',
       'PCF_VIOL_CATEGORY', 'PCF_VIOLATION', 'PCF_VIOL_SUBSECTION',
       'HIT_AND_RUN', 'TYPE_OF_COLLISION', 'MVIW', 'PED_ACTION',
       'ROAD_SURFACE', 'ROAD_COND_1', 'ROAD_COND_2', 'LIGHTING',
       'CONTROL_DEVICE', 

,CASE_ID,PARTY_NUMBER,PARTY_TYPE,AT_FAULT,PARTY_SEX,PARTY_AGE,PARTY_SOBRIETY,PARTY_DRUG_PHYSICAL,DIR_OF_TRAVEL,PARTY_SAFETY_EQUIP_1,...,VEHICLE_YEAR,VEHICLE_MAKE,STWD_VEHICLE_TYPE,CHP_VEH_TYPE_TOWING,CHP_VEH_TYPE_TOWED,RACE,INATTENTION,SPECIAL_INFO_F,SPECIAL_INFO_G,ACCIDENT_YEAR
0,6291500,1,4,Y,M,72,A,-,W,-,...,NaN,-,L,4.0,NaN,A,NaN,-,-,2014
1,6291500,2,4,N,M,38,A,-,E,-,...,NaN,-,L,4.0,NaN,H,NaN,-,-,2014
2,6291600,1,4,Y,M,57,B,-,E,-,...,NaN,-,L,4.0,NaN,W,NaN,-,-,2014
3,6291600,2,1,N,M,48,A,-,S,M,...,2001.0,JEEP,A,7.0,NaN,H,NaN,-,-,2014
4,6291825,1,4,Y,M,55,D,-,W,-,...,NaN,-,L,4.0,NaN,W,NaN,-,-,2014


Parties shape: (4097, 34)
Index(['CASE_ID', 'PARTY_NUMBER', 'PARTY_TYPE', 'AT_FAULT', 'PARTY_SEX',
       'PARTY_AGE', 'PARTY_SOBRIETY', 'PARTY_DRUG_PHYSICAL', 'DIR_OF_TRAVEL',
       'PARTY_SAFETY_EQUIP_1', 'PARTY_SAFETY_EQUIP_2', 'FINAN_RESPONS',
       'SP_INFO_1', 'SP_INFO_2', 'SP_INFO_3', 'OAF_VIOLATION_CODE',
       'OAF_VIOL_CAT', 'OAF_VIOL_SECTION', 'OAF_VIOLATION_SUFFIX', 'OAF_1',
       'OAF_2', 'PARTY_NUMBER_KILLED', 'PARTY_NUMBER_INJURED', 'MOVE_PRE_ACC',
       'VEHICLE_YEAR', 'VEHICLE_MAKE', 'STWD_VEHICLE_TYPE',
       'CHP_VEH_TYPE_TOWING', 'CHP_VEH_TYPE_TOWED', 'RACE', 'INATTENTION',
       'SPECIAL_INFO_F', 'SPECIAL_INFO_G', 'ACCIDENT_YEAR'],
      dtype='object')


,CASE_ID,PARTY_NUMBER,VICTIM_NUMBER,VICTIM_ROLE,VICTIM_SEX,VICTIM_AGE,VICTIM_DEGREE_OF_INJURY,VICTIM_SEATING_POSITION,VICTIM_SAFETY_EQUIP_1,VICTIM_SAFETY_EQUIP_2,VICTIM_EJECTED,COUNTY,CITY,ACCIDENT_YEAR
0,6291500,2,1,4,M,38,3,1,W,-,0,SAN DIEGO,SAN DIEGO,2014
1,6291600,1,1,4,M,57,1,1,P,V,1,SAN DIEGO,SAN DIEGO,2014
2,6291825,1,1,4,M,55,1,1,P,V,1,SAN DIEGO,SAN DIEGO,2014
3,6328108,1,1,4,M,41,3,1,-,V,1,SAN DIEGO,SAN DIEGO,2014
4,6337228,1,1,4,M,50,3,9,-,W,3,SAN DIEGO,SAN DIEGO,2014


Victims shape: (2723, 14)
Index(['CASE_ID', 'PARTY_NUMBER', 'VICTIM_NUMBER', 'VICTIM_ROLE', 'VICTIM_SEX',
       'VICTIM_AGE', 'VICTIM_DEGREE_OF_INJURY', 'VICTIM_SEATING_POSITION',
       'VICTIM_SAFETY_EQUIP_1', 'VICTIM_SAFETY_EQUIP_2', 'VICTIM_EJECTED',
       'COUNTY', 'CITY', 'ACCIDENT_YEAR'],
      dtype='object')


The field that is common for both tables is the 'CASE_ID' column, which is "the unique identifier of the collision report" ([TIMS](https://tims.berkeley.edu/help/SWITRS.php#Codebook)). Therefore we are joining on this column.

In [206]:
collisions_and_party = pd.merge(collisions, parties, on="CASE_ID")

# make sure that it's bike accidents only
cp_bike_only = collisions_and_party[collisions_and_party['BICYCLE_ACCIDENT']== 'Y']
display(cp_bike_only.head())
print(cp_bike_only.shape)

,CASE_ID,ACCIDENT_YEAR_x,PROC_DATE,JURIS,COLLISION_DATE,COLLISION_TIME,OFFICER_ID,REPORTING_DISTRICT,DAY_OF_WEEK,CHP_SHIFT,...,VEHICLE_YEAR,VEHICLE_MAKE,STWD_VEHICLE_TYPE,CHP_VEH_TYPE_TOWING,CHP_VEH_TYPE_TOWED,RACE,INATTENTION,SPECIAL_INFO_F,SPECIAL_INFO_G,ACCIDENT_YEAR_y
0,6291500,2014,2014-10-29,3711,2014-08-06,1624,4653,NaN,3,5,...,NaN,-,L,4.0,NaN,A,NaN,-,-,2014
1,6291500,2014,2014-10-29,3711,2014-08-06,1624,4653,NaN,3,5,...,NaN,-,L,4.0,NaN,H,NaN,-,-,2014
2,6291600,2014,2016-01-12,3711,2014-10-07,2312,5051,NaN,2,5,...,NaN,-,L,4.0,NaN,W,NaN,-,-,2014
3,6291600,2014,2016-01-12,3711,2014-10-07,2312,5051,NaN,2,5,...,2001.0,JEEP,A,7.0,NaN,H,NaN,-,-,2014
4,6291825,2014,2015-01-09,3711,2014-11-09,1729,6366,NaN,7,5,...,NaN,-,L,4.0,NaN,W,NaN,-,-,2014


(4097, 113)


In [207]:
# this part checks what years these accidents were recorded in order to check if the CASE_ID is 
# the same format for all years. According to TIMS, there was a 19 digit code prior to 2002

collisions_and_party['ACCIDENT_YEAR_x'].unique()

array([2014, 2016, 2015, 2017, 2018])

### Clean spatial data as needed and create a point layer of collisions

In [208]:
# Cleaned the dataframes to only include what we thought were relevant columns
columns_collision_party = ['CASE_ID', 'WEATHER_1','TYPE_OF_COLLISION','COLLISION_DATE', 'COLLISION_TIME', 'DAY_OF_WEEK', 'COLLISION_SEVERITY', 'PARTY_COUNT', 'PCF_VIOL_CATEGORY', 'BICYCLE_ACCIDENT', 'ALCOHOL_INVOLVED', 'COUNT_BICYCLIST_KILLED', 'COUNT_BICYCLIST_INJURED', 'LATITUDE', 'LONGITUDE', 'POINT_X', 'POINT_Y', 'CITY', 'PARTY_NUMBER', 'PARTY_TYPE', 'PARTY_SOBRIETY', 'PARTY_DRUG_PHYSICAL', 'STWD_VEHICLE_TYPE', 'AT_FAULT']
cp_bike_only = cp_bike_only[columns_collision_party]

#Impute Nans with 0s in Alcohol_involved column - answer the question on which parts of the city you're
#more likely to get into an alcohol-related accident

def onehotencode(row):
    if row == 'Y':
        return 1
    else:
        return 0
cp_bike_only['ALCOHOL_INVOLVED'] = cp_bike_only['ALCOHOL_INVOLVED'].apply(onehotencode)


In [209]:
# impute any null values with existing values in the POINT_X and POINT_Y columns
# We did this because most of the longitude and latitude values were null
cp_bike_only['LATITUDE'] = cp_bike_only['LATITUDE'].fillna(cp_bike_only['POINT_X'])
cp_bike_only['LONGITUDE'] = cp_bike_only['LONGITUDE'].fillna(cp_bike_only['POINT_Y'])

In [210]:
cp_bike_only.info() # check if LATITUDE and LONGITUDE have the same amount of null values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4097 entries, 0 to 4096
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   CASE_ID                  4097 non-null   int64  
 1   WEATHER_1                4097 non-null   object 
 2   TYPE_OF_COLLISION        4097 non-null   object 
 3   COLLISION_DATE           4097 non-null   object 
 4   COLLISION_TIME           4097 non-null   int64  
 5   DAY_OF_WEEK              4097 non-null   int64  
 6   COLLISION_SEVERITY       4097 non-null   int64  
 7   PARTY_COUNT              4097 non-null   int64  
 8   PCF_VIOL_CATEGORY        4097 non-null   object 
 9   BICYCLE_ACCIDENT         4097 non-null   object 
 10  ALCOHOL_INVOLVED         4097 non-null   int64  
 11  COUNT_BICYCLIST_KILLED   4097 non-null   int64  
 12  COUNT_BICYCLIST_INJURED  4097 non-null   int64  
 13  LATITUDE                 3710 non-null   float64
 14  LONGITUDE               

In [211]:
# Assessment of missingness
from scipy.stats import ks_2samp
import numpy as np

def perm4missing(cp_bike_only, col):
    """ Conducts a KS-Statistic permutation test. The KS test is used to identify whether
    two distributions are from the same continuous distribution. Using the KS statistic test, we were able 
    to create two samples of data (one which contains the distribution of a column where LATITUDE is null 
    and the other which contains the distribution of a column where LATITUDE is not null). After generating 
    multiple samples through the 1000 simulations, we compared to our observed statistic which was the 
    ks_2samp of our initial two independent samples"""
    
    filtered = cp_bike_only.assign(_null=cp_bike_only['LATITUDE'].isnull())
    no_null = filtered.loc[filtered['_null'] == False , col]
    is_null = filtered.loc[filtered['_null'] == True, col]
    obs = ks_2samp(no_null, is_null).statistic
    n_repetitions = 500

    ks_list = []
    for _ in range(n_repetitions):
    
        # shuffle the comparison column
        shuffled_col = (filtered[col].sample(replace=False, frac=1).reset_index(drop=True))
    
        # put them in a table
        shuffled = (filtered.assign(**{'Shuffled': shuffled_col,}))
    
        # compute the KS
        grps = shuffled.groupby('_null')['Shuffled']
        ks = ks_2samp(grps.get_group(True), grps.get_group(False)).statistic
    
        ks_list.append(ks)
    
    ks_list = np.array(ks_list)

    pval = np.count_nonzero(ks_list > obs)/len(ks_list)
    
    return pval

In [212]:
pval = perm4missing(cp_bike_only, "COLLISION_DATE")
print("COLLISION_DATE: " + str(pval))

COLLISION_DATE: 0.0


In order to check why there may be zip codes mentioned in the SWITRS data that don't have associated polygons (aka have null coordinate values), we did an assessment of missingness test to try and figure out if the null values in the "LATITUDE" column were dependent on the values of another column. We performed the assessment of missingness on the "LATITUDE" column because that column's values were how we determined polygon shapes. We found that the "LATITUDE" column was highly dependent on the "COLLISION_DATE" column since the p-value was less than 0.05. This indicates that the null values in "LATITUDE" have high dependency on the values in "COLLISION_DATE". Because of this, we're thinking that the null values in in the column may be due to GPS tracking machines being down on certain dates, causing the tracking of a collision to be non-existent. In order to deal with this situation, we'll remove any values with null coordinates. Further explanation is given below. 

In [213]:
#Drop the rows with null coordinates because they might not map properly 
dropped_df = cp_bike_only[cp_bike_only['LATITUDE'].isna() == False]

print(dropped_df.shape)
dropped_df.head()

(3710, 24)


,CASE_ID,WEATHER_1,TYPE_OF_COLLISION,COLLISION_DATE,COLLISION_TIME,DAY_OF_WEEK,COLLISION_SEVERITY,PARTY_COUNT,PCF_VIOL_CATEGORY,BICYCLE_ACCIDENT,...,LONGITUDE,POINT_X,POINT_Y,CITY,PARTY_NUMBER,PARTY_TYPE,PARTY_SOBRIETY,PARTY_DRUG_PHYSICAL,STWD_VEHICLE_TYPE,AT_FAULT
0,6291500,A,H,2014-08-06,1624,3,3,2,22,Y,...,32.911904,-117.103594,32.911904,SAN DIEGO,1,4,A,-,L,Y
1,6291500,A,H,2014-08-06,1624,3,3,2,22,Y,...,32.911904,-117.103594,32.911904,SAN DIEGO,2,4,A,-,L,N
2,6291600,A,H,2014-10-07,2312,2,1,2,22,Y,...,32.754250,-117.205080,32.754250,SAN DIEGO,1,4,B,-,L,Y
3,6291600,A,H,2014-10-07,2312,2,1,2,22,Y,...,32.754250,-117.205080,32.754250,SAN DIEGO,2,1,A,-,A,N
4,6291825,A,H,2014-11-09,1729,7,1,1,03,Y,...,32.750301,-117.136246,32.750301,SAN DIEGO,1,4,D,-,L,Y


We decided to drop coordinates with any null values in order to avoid a lot of mapping issues. Since we didn't see too large of a decrease in coordinates (4097 collisions w/ party --> 3710 collisions w/ party) we decided to keep it filtered this way.

In [214]:
#Convert into SEDF from GeoDataframe
sdf = pd.DataFrame.spatial.from_xy(dropped_df , x_column = 'LATITUDE', y_column='LONGITUDE')
print(sdf.shape)
display(sdf.head(2))

#Convert into Feature Layer
collision_point_fl = sdf.spatial.to_featurelayer(title ='Bike Collision Points', gis=gis, tags="bikes")

(3710, 25)


,CASE_ID,WEATHER_1,TYPE_OF_COLLISION,COLLISION_DATE,COLLISION_TIME,DAY_OF_WEEK,COLLISION_SEVERITY,PARTY_COUNT,PCF_VIOL_CATEGORY,BICYCLE_ACCIDENT,...,POINT_X,POINT_Y,CITY,PARTY_NUMBER,PARTY_TYPE,PARTY_SOBRIETY,PARTY_DRUG_PHYSICAL,STWD_VEHICLE_TYPE,AT_FAULT,SHAPE
0,6291500,A,H,2014-08-06,1624,3,3,2,22,Y,...,-117.103594,32.911904,SAN DIEGO,1,4,A,-,L,Y,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117..."
1,6291500,A,H,2014-08-06,1624,3,3,2,22,Y,...,-117.103594,32.911904,SAN DIEGO,2,4,A,-,L,N,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117..."


Feature Service Link for Collisions Points: https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/afa9db/FeatureServer?token=xjbxWAVtGTiYl376vUNv_DFq8fTfPYtPonpCcBXmf8Lg8KehC_E8KkIjao1KfBFmaD4SrOzLBCiA5IkYEYCVkJ6foW2r6w77SrL0UpvjzNPPesTt3ONnxuZbLPfaDDjda_oc9utmm0XYTUxu_3lFgv0tdj4f12ZXMgNClgSSWbgXww76yQ_uOUFIvvIk2qdTbwLkvZ-pZ62SFhxBzybw1utfPMMrDyAUNx7Zr-FbKFkp8xDyO1IXHlB5kBFR9SnClV8MKrsR7gwMH7IfS_PdbQ.

Feature Service ItemId: 59ee5c4b8b444db3a115a66519a027a1

In [215]:
# Generate map that visualizes all of the collisions in SD city
collisions_in_sd = gis.map('San Diego, CA')
collisions_in_sd.add_layer(collision_point_fl)
collisions_in_sd

MapView(layout=Layout(height='400px', width='100%'))

In [216]:
# Gather bike route data, define what you mean to be on a bike route,  and figure out, 
# for each collision, whether it happened on a bike route or not. 

m = gis.map('San Diego, CA')

# Get bike route feature layer and create buffers around the bike routes
# buffer_bike_routes = create_buffers(bike_routes_fl,
#                                distances=[10],
#                                ring_type='Rings',
#                                units='Feet',
#                                output_name="Buffer_Bike_Routes_10ft_final")

buffer_bike_routes = gis.content.get('7a4e5425e8ae4ea3abb4fa97186fa816')
buffer_routes_fl = buffer_bike_routes.layers[0]
m.add_layer(buffer_routes_fl)

# Find intersecting 
# intersecting_collisions = join_features(target_layer= collision_point_fl, 
#                                         join_layer= buffer_routes_fl,
#                                         spatial_relationship='intersects',
#                                         output_name= 'Intersecting_Collision_with_Route_Buffer_final')

intersecting_collisions = gis.content.get('dd7d6b4be815469ab5778fb0a16caf9a')
intersecting_collisions_fl = intersecting_collisions.layers[0]
m.add_layer(intersecting_collisions_fl)


Upon Googling how wide an average bike lane is, we got the answer 4 ft ([source](https://www.google.com/search?rlz=1C5CHFA_enUS807US807&sxsrf=ALeKk02B4ktRs_3w2OI57SkRNN3XRsyFqg%3A1605361210462&ei=Ot6vX9rLG_zF0PEPkoK4iAU&q=how+wide+are+bike+lanes+in+america&oq=how+wide+are+bike+lanes+in+america&gs_lcp=CgZwc3ktYWIQAzIHCCEQChCgAToECAAQRzoJCAAQyQMQFhAeOgYIABAWEB46CAghEBYQHRAeOgUIIRCgAToFCCEQqwJQ9ThYgkFgmUFoAHACeACAAXWIAdQIkgEEMC4xMJgBAKABAaoBB2d3cy13aXrIAQjAAQE&sclient=psy-ab&ved=0ahUKEwja0L3FlILtAhX8IjQIHRIBDlEQ4dUDCA0&uact=5)). In order to account for any miscalculations in the bike collision coordinates, we decided to create 10 ft buffers around the bike routes. We then found any intersecting collision coordinates with those bike routes with buffers.

In [217]:
m

MapView(layout=Layout(height='400px', width='100%'))

In [218]:
# bikes that were in a collision in a bike lane
clipped_buffer_fs = intersecting_collisions_fl.query()
print(clipped_buffer_fs.sdf.shape)
clipped_buffer_fs.sdf.head()

(1553, 35)


,OBJECTID,Join_Count,TARGET_FID,CASE_ID,COLLISION_,COLLISION1,DAY_OF_WEE,COLLISION2,PARTY_COUN,PCF_VIOL_C,...,RD20FULL,Jurisdicti,ROUTE,Route_Clas,Max_Elev,Shape_Leng,BUFF_DIST,ORIG_FID,AnalysisArea,SHAPE
0,1,3,12,6560089,2014-07-05,2035,6,3,2,08,...,ROBINSON AV,San Diego,3,Bike Route,310.818474,4712.994900,10,2194,0.003392,"{""x"": -13042237.1923, ""y"": 3861762.467699997, ..."
1,2,3,13,6560089,2014-07-05,2035,6,3,2,08,...,ROBINSON AV,San Diego,3,Bike Route,310.818474,4712.994900,10,2194,0.003392,"{""x"": -13042237.1923, ""y"": 3861762.467699997, ..."
2,3,4,14,6560090,2014-07-02,930,3,4,2,08,...,PARK BL,San Diego,15,Bikeways Coming Soon,310.818474,2016.624031,10,1964,0.001458,"{""x"": -13040802.442200001, ""y"": 3861000.253700..."
3,4,4,15,6560090,2014-07-02,930,3,4,2,08,...,PARK BL,San Diego,15,Bikeways Coming Soon,310.818474,2016.624031,10,1964,0.001458,"{""x"": -13040802.442200001, ""y"": 3861000.253700..."
4,5,1,16,6560164,2014-07-05,9,6,3,2,08,...,EAST MISSION BAY DR,San Diego,3,Bike Route,23.070590,8983.656945,10,816,0.006456,"{""x"": -13047668.6518, ""y"": 3866233.423600003, ..."


In [251]:
# check the number of collisions without duplicates
clipped_buffer_fs.sdf.groupby("CASE_ID").count().shape[0]

868

## Part 2

* When you are asked to summarize collisions by zip codes and geoenrich, you may want to look at variables in geoenrichment categories such as businesses, GroceryAlcoholicBeverages, LeisureActivitiesLifestyle, food. You may also use the businesses data you worked with in MP2 and aggregated business locations (e.g., bars) by zip codes.
* To find a zip codes feature layer or shapefile, search external content on ArcGIS Online. Note that there may be zip codes mentioned in the SWITRS data that don't have associated polygons (try to figure out why, and add comments in the notebook to justify your solutions.)
* For additional analysis, to address the questions at the top of the MP3 notebook, you may use a data analysis technique (eg regression) you learned in an earlier course. For example, you may use scikit-learn to build a model explaining severity of injuries as dependent on impairment, type of collision, and location.
Please add up to half a page describing your findings (in the last cell, which is marked with 5 points.)

In [219]:
# regression/classification imports 
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

In [220]:
#(5 points)
# Summarize collisions by zip codes and geoenrich.

from arcgis.geoenrichment import *

# Your code goes here

collisions['LATITUDE'] = collisions['LATITUDE'].fillna(collisions['POINT_X'])
collisions['LONGITUDE'] = collisions['LONGITUDE'].fillna(collisions['POINT_Y'])
dropped_null_collisions = collisions[collisions['LATITUDE'].isna() == False]

# according to metadata this zip code layers only contains values for 123 zip codes in SD county
sd_zip_code = gis.content.get('15e4d8d850674b0a8293e4d91978ae95')
sd_zip_code_fl = sd_zip_code.layers[0]

from arcgis.features import summarize_data

coll_sdf = pd.DataFrame.spatial.from_xy(dropped_null_collisions , x_column = 'LATITUDE', y_column='LONGITUDE')
collision_point_fl_no_duplicates = coll_sdf.spatial.to_featurelayer(title ='Bike_Collision_Points_No_Duplicates', gis=gis) 

agg_points_fc = summarize_data.aggregate_points(point_layer = collision_point_fl_no_duplicates,
                                                polygon_layer = sd_zip_code_fl,
                                                keep_boundaries_with_no_points = True,
                                                summary_fields = ['CASE_ID sum'])['aggregated_layer']

In [221]:
agg_points_fc.query().sdf['ZIP'] # this layers is aggregating by the number of county zip codes

0      92056
1      92161
2      92058
3      92134
4      92058
       ...  
118    92055
119    92028
120    92065
121    92036
122    92004
Name: ZIP, Length: 123, dtype: int64

In [222]:
# maps that helps sanity check how many collisions were in a certain zip code
zip_code_map = gis.map('San Diego, CA')
zip_code_map.add_layer(sd_zip_code_fl)
zip_code_map.add_layer(intersecting_collisions_fl)
zip_code_map.add_layer(collision_point_fl_no_duplicates)
zip_code_map

MapView(layout=Layout(height='400px', width='100%'))

In order to sanity check the the aggregation we did below, we printed this map and zoomed into certain zip codes to check how many orange bike accidents happened in that zip code. This helps validate that the number of bike collisions we got per zip code was correct.

In [223]:
# Summarize about how any collisions were in each zip code

zip_collision_list = agg_points_fc.query().sdf['ZIP']
zip_collision_count_list = agg_points_fc.query().sdf['Point_Count']

zip_collision_counts_dict = {}
for i in range(len(zip_collision_list)):
    zip_collision_counts_dict[zip_collision_list[i]] = [zip_collision_count_list[i]]

sorted_keys = sorted(zip_collision_counts_dict.keys())

for i in sorted_keys:
    print(i, zip_collision_counts_dict[i])

91901 [0]
91902 [0]
91905 [0]
91906 [0]
91910 [0]
91911 [2]
91913 [0]
91914 [0]
91915 [0]
91916 [0]
91917 [0]
91931 [0]
91932 [0]
91934 [0]
91935 [0]
91941 [0]
91942 [0]
91945 [0]
91948 [0]
91950 [1]
91962 [0]
91963 [0]
91977 [0]
91978 [0]
91980 [0]
92003 [0]
92004 [0]
92007 [0]
92008 [0]
92009 [0]
92010 [0]
92011 [0]
92014 [11]
92019 [0]
92020 [1]
92021 [0]
92024 [0]
92025 [0]
92026 [0]
92027 [0]
92028 [0]
92029 [0]
92036 [0]
92037 [99]
92040 [0]
92054 [0]
92055 [0]
92056 [0]
92057 [0]
92058 [0]
92059 [0]
92060 [0]
92061 [0]
92064 [0]
92065 [0]
92066 [0]
92067 [1]
92069 [0]
92070 [0]
92071 [0]
92075 [0]
92078 [0]
92081 [0]
92082 [0]
92083 [0]
92084 [0]
92086 [0]
92091 [0]
92093 [14]
92096 [0]
92101 [233]
92102 [75]
92103 [77]
92104 [110]
92105 [129]
92106 [37]
92107 [67]
92108 [40]
92109 [243]
92110 [94]
92111 [63]
92113 [57]
92114 [30]
92115 [85]
92116 [45]
92117 [57]
92118 [1]
92119 [11]
92120 [20]
92121 [37]
92122 [31]
92123 [48]
92124 [13]
92126 [69]
92127 [10]
92128 [17]
92129 [2

In [224]:
# convert to dataframe
zip_collision_counts_df = pd.DataFrame.from_dict(
    zip_collision_counts_dict).T.rename({0 :'Number_of_Bike_Collisions'}, axis='columns')
display(zip_collision_counts_df.head(3))

,Number_of_Bike_Collisions
92056,0
92161,0
92058,0


In [225]:
# prepare to geoenrich

usa = Country.get('USA')
df = usa.data_collections # check out all of the data collections for the USA
display(df[df.index.values == "Disposable_Income_Profile_rep"]) # DIBASE_CY --> base disposable income
display(df[df.index.values == "retailmarketplace"]) 
# retailmarketplace.RETBUS4453 --> 2017 Businesses: Beer/Wine/Liquor Stores (NAICS 4453)
# retailmarketplace.RETBUS7224 --> 2017 Businesses: Drinking Places-Alcohol (NAICS 7224)

# check if any alcohol is served at the establishments
relevant_columns = ['DIBASE_CY']
for i in df[df.index.values == "retailmarketplace"].analysisVariable:
    if 'BUS7224' in i or 'BUS4453' in i:
        relevant_columns.append(i[18:])
        
print("Relevant Columns: " + str(relevant_columns))


,analysisVariable,alias,fieldCategory,vintage
dataCollectionID,,,,
Disposable_Income_Profile_rep,Disposable_Income_Profile_rep.DIBASE_CY,2020 Disposable Income Base,2020 Disposable Income by Age (Esri),2020
Disposable_Income_Profile_rep,Disposable_Income_Profile_rep.DIA15BASCY,2020 Disposable Inc Base: HHr 15-24,2020 Disposable Income by Age (Esri),2020
Disposable_Income_Profile_rep,Disposable_Income_Profile_rep.DIA25BASCY,2020 Disposable Inc Base: HHr 25-34,2020 Disposable Income by Age (Esri),2020
Disposable_Income_Profile_rep,Disposable_Income_Profile_rep.DIA35BASCY,2020 Disposable Inc Base: HHr 35-44,2020 Disposable Income by Age (Esri),2020
Disposable_Income_Profile_rep,Disposable_Income_Profile_rep.DIA45BASCY,2020 Disposable Inc Base: HHr 45-54,2020 Disposable Income by Age (Esri),2020
Disposable_Income_Profile_rep,Disposable_Income_Profile_rep.DIA55BASCY,2020 Disposable Inc Base: HHr 55-64,2020 Disposable Income by Age (Esri),2020
Disposable_Income_Profile_rep,Disposable_Income_Profile_rep.DIA65BASCY,2020 Disposable Inc Base: HHr 65-74,2020 Disposable Income by Age (Esri),2020
Disposable_Income_Profile_rep,Disposable_Income_Profile_rep.DIA75BASCY,2020 Disposable Inc Base: HHr 75+,2020 Disposable Income by Age (Esri),2020


,analysisVariable,alias,fieldCategory,vintage
dataCollectionID,,,,
retailmarketplace,retailmarketplace.RTSALESTOT,2017 Total Retail:Sales,2017 Retail MarketPlace,2017
retailmarketplace,retailmarketplace.RETPOTTOT,2017 Total Retail:Pot,2017 Retail MarketPlace,2017
retailmarketplace,retailmarketplace.LSFRETTOT,2017 Total Retail:L/S,2017 Retail MarketPlace,2017
retailmarketplace,retailmarketplace.RETBUSTOT,2017 Total Retail:Bus,2017 Retail MarketPlace,2017
retailmarketplace,retailmarketplace.RTTRDSALES,2017 Retail Trade:Sales,2017 Retail MarketPlace,2017
...,...,...,...,...
retailmarketplace,retailmarketplace.RETBUS7224,2017 Drinking Places-Alcohol:Bus,2017 Retail MarketPlace,2017
retailmarketplace,retailmarketplace.RSALES7225,2017 Restaurants/Other Eating Places:Sales,2017 Retail MarketPlace,2017
retailmarketplace,retailmarketplace.RETPOT7225,2017 Restaurants/Other Eating Places:Pot,2017 Retail MarketPlace,2017


Relevant Columns: ['DIBASE_CY', 'RETBUS4453', 'RETBUS7224']


During geoenriching, we found that the "Disposable_Income_Profile_rep" and the "retailmarketplace" data collections helped us find relevant features for future regression. We looked at datasets also given in the instructions (EX: businesses, GroceryAlcoholicBeverages, LeisureActivitiesLifestyle, food), but we found that these were a little more along the line of what we wanted to use as features. The columns that we extracted are explained in the comments in the cell above this. We chose to see the disposable income of a zip code in order to see if maybe higher disposable incomes lead to a higher number of alcohol stores and bars for people to spend their money at. We also extracted the number of businesses of Beer/Wine/Liquor Stores and Alcohol Drinking Places (aka bars) via NAICS codes. We did this so that we could see if there were a higher number of alchol related bike crashes as a result of a higher number of both of those kinds of stores. 

In [226]:
# initialize geoenriched features
zip_collision_counts_df['Disposable_Income'] = 0
zip_collision_counts_df['Number_of_Alcohol_Stores'] = 0
zip_collision_counts_df['Number_of_Bars'] = 0
zip_collision_counts_df.head(2)

,Number_of_Bike_Collisions,Disposable_Income,Number_of_Alcohol_Stores,Number_of_Bars
92056,0,0,0,0
92161,0,0,0,0


In [227]:
# geoenrich any zip code
for i in zip_collision_list:
    zip_stats = enrich(study_areas=[str(i)], data_collections=['Disposable_Income_Profile_rep', 
                                                'retailmarketplace'])[relevant_columns]
    zip_collision_counts_df.loc[i, 'Disposable_Income'] = zip_stats['DIBASE_CY'][0]
    zip_collision_counts_df.loc[i, 'Number_of_Alcohol_Stores'] = zip_stats['RETBUS4453'][0]
    zip_collision_counts_df.loc[i, 'Number_of_Bars'] = zip_stats['RETBUS7224'][0]

In [228]:
# filter in only san diego city zip codes
sd_city_zip_codes = [91911, 91914, 91915, 91932, 91942, 91945, 91950, 92014, 92025, 92027, 92029, 92037, 92064, 92065, 92067, 92071, 92075, 92101, 92102, 92103, 92104, 92105, 92106, 92107, 92108, 92109, 92110, 92111, 92113, 92114, 92115, 92116, 92117, 92118, 92119, 92120, 92121, 92122, 92123, 92124, 92126, 92127, 92128, 92129, 92130, 92131, 92132, 92134, 92135, 92139, 92140, 92145, 92147, 92154, 92173]
zip_collision_counts_df = zip_collision_counts_df[zip_collision_counts_df.index.isin(sd_city_zip_codes)]

# display geoenriched zip codes dataframe
zip_collision_counts_df.head(3)

,Number_of_Bike_Collisions,Disposable_Income,Number_of_Alcohol_Stores,Number_of_Bars
92134,0,14180,8,10
92140,0,2259,5,10
92104,110,27105,12,17


In [229]:
print("The number of zip codes in SD city: " + str(len(sd_city_zip_codes)))
print("The number of zip codes after filtering with the counties dataframe: " + str(zip_collision_counts_df.shape[0]))

The number of zip codes in SD city: 55
The number of zip codes after filtering with the counties dataframe: 53


We tried to filter the county's zip code data frame in order to get only zip codes in SD city ([source](https://www.city-data.com/zipmaps/San-Diego-California.html)). According to the metadata of the feature layer for SD county's zip code data, this report "does not represent all zip codes that may be used in the county." This means that not all zip codes are represented in this layer because some features cannot be represented as polygon, as stated in the metadata as well. This means that when we filter the orginal 123 zip codes to only take into account SD city zip codes, some zip codes may not show up in the dataframe. This is proven with the cell above since we got data that indicates there are 55 counties in SD city, but only 53 of them show up after filtering the 123 data points from the original SD county zip code dataframe. Since this data was estimated from SanGIS, we chose to keep the data despite 2 zip codes being dropped. Further explanation included below.

In [230]:
zip_codes_123 = list(zip_collision_counts_df.index)
[x for x in sd_city_zip_codes if x not in zip_codes_123]

[92132, 92147]

These are the 2 zip codes that were were not in the original 123 zip code layer for San Diego county. Upon Googling these to zip codes, they seem to be very small areas that are affiliated with docking areas for boats ([source](https://www.google.com/maps/place/San+Diego,+CA+92132/@32.7133894,-117.1750417,1320m/data=!3m1!1e3!4m5!3m4!1s0x80d95358bfac6f09:0x1c8085ded1e6bf2!8m2!3d32.7150419!4d-117.1724288) for 92132, [source](https://www.google.com/maps/place/San+Diego,+CA+92147/@32.7249667,-117.2224506,1584m/data=!3m1!1e3!4m5!3m4!1s0x80deab0abdc71c41:0xc300a1c30346a1eb!8m2!3d32.7255292!4d-117.218372) for 92147). After reviewing the map above that maps out the zip codes in blue, it seems like these two zip codes were dissolved to be incorporated into the larger zip code surrounding these small areas

# Performing Additional Analysis
### Are you more or less likely to be injured in a bike-related accident if you are on a bike path or not?
- __How would the results differ for injuries of different severity?__
- __How would the results differ for injuries of different zip codes?__
- __How would the results differ for injuries by the type of other party in the accident?__

### In which parts of the city are you more likely to get into an alcohol-related accident?
- __How can you explain that using additional statistics by zip codes, for example, by the number of alcohol-serving bars within the zip code?__

In [232]:
all_collisions = sdf # sdf of all of the collisions (including multiple rows for the party)
display(all_collisions.head(2))
intersecting_collisions = clipped_buffer_fs.sdf # sdf of all of the bike collisions that intersect with a bike lane
display(intersecting_collisions.head(2))

,CASE_ID,WEATHER_1,TYPE_OF_COLLISION,COLLISION_DATE,COLLISION_TIME,DAY_OF_WEEK,COLLISION_SEVERITY,PARTY_COUNT,PCF_VIOL_CATEGORY,BICYCLE_ACCIDENT,...,POINT_X,POINT_Y,CITY,PARTY_NUMBER,PARTY_TYPE,PARTY_SOBRIETY,PARTY_DRUG_PHYSICAL,STWD_VEHICLE_TYPE,AT_FAULT,SHAPE
0,6291500,A,H,2014-08-06,1624,3,3,2,22,Y,...,-117.103594,32.911904,SAN DIEGO,1,4,A,-,L,Y,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117..."
1,6291500,A,H,2014-08-06,1624,3,3,2,22,Y,...,-117.103594,32.911904,SAN DIEGO,2,4,A,-,L,N,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117..."


,OBJECTID,Join_Count,TARGET_FID,CASE_ID,COLLISION_,COLLISION1,DAY_OF_WEE,COLLISION2,PARTY_COUN,PCF_VIOL_C,...,RD20FULL,Jurisdicti,ROUTE,Route_Clas,Max_Elev,Shape_Leng,BUFF_DIST,ORIG_FID,AnalysisArea,SHAPE
0,1,3,12,6560089,2014-07-05,2035,6,3,2,08,...,ROBINSON AV,San Diego,3,Bike Route,310.818474,4712.9949,10,2194,0.003392,"{""x"": -13042237.1923, ""y"": 3861762.467699997, ..."
1,2,3,13,6560089,2014-07-05,2035,6,3,2,08,...,ROBINSON AV,San Diego,3,Bike Route,310.818474,4712.9949,10,2194,0.003392,"{""x"": -13042237.1923, ""y"": 3861762.467699997, ..."


##### Question 1

__Are you more or less likely to be injured in a bike-related accident if you are on a bike path or not?__

In [233]:
#Converting to the same spatial reference coordinates
zip_code_sdf4326 = sd_zip_code_fl.query(out_sr = 4326).sdf # sd county zip codes
intersecting_collisions_4326 = intersecting_collisions_fl.query(out_sr = 4326).sdf # collisions that intersect with a bike lane

#Joining the collisions with san diego zipcodes
all_collision_with_zip = all_collisions.spatial.join(zip_code_sdf4326) 
display(all_collision_with_zip.head())
intersecting_collision_with_zip = intersecting_collisions_4326.spatial.join(zip_code_sdf4326)
display(intersecting_collision_with_zip.head())

num_cols_all = all_collision_with_zip.groupby('CASE_ID').count().shape[0] #Number of all collisions grouped by zipcodes
num_cols_in_bikelane = intersecting_collision_with_zip.groupby('CASE_ID').count().shape[0] #Number of collision in a bikelane grouped by zipcodes
num_cols_notin_bikelane = num_cols_all - num_cols_in_bikelane #Number of collision not in a bikelane grouped by zipcodes
print('Collisions in bike lanes: ' + str(num_cols_in_bikelane))
print('Collisions outside of bike lanes: ' + str(num_cols_notin_bikelane))

,CASE_ID,WEATHER_1,TYPE_OF_COLLISION,COLLISION_DATE,COLLISION_TIME,DAY_OF_WEEK,COLLISION_SEVERITY,PARTY_COUNT,PCF_VIOL_CATEGORY,BICYCLE_ACCIDENT,...,AT_FAULT,SHAPE,index_right,FID,ZIP,COMMUNITY,SHAPE_STAr,SHAPE_STLe,Shape__Area,Shape__Length
0,6291500,A,H,2014-08-06,1624,3,3,2,22,Y,...,Y,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117...",108,109,92131,San Diego,3.911265e+08,137602.571318,5.171307e+07,50006.616039
1,6291500,A,H,2014-08-06,1624,3,3,2,22,Y,...,N,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117...",108,109,92131,San Diego,3.911265e+08,137602.571318,5.171307e+07,50006.616039
2,6417345,A,H,2014-03-12,740,3,3,1,18,Y,...,N,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117...",108,109,92131,San Diego,3.911265e+08,137602.571318,5.171307e+07,50006.616039
3,7053051,A,D,2015-08-28,830,5,3,2,12,Y,...,Y,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117...",108,109,92131,San Diego,3.911265e+08,137602.571318,5.171307e+07,50006.616039
4,7053051,A,D,2015-08-28,830,5,3,2,12,Y,...,N,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117...",108,109,92131,San Diego,3.911265e+08,137602.571318,5.171307e+07,50006.616039


,OBJECTID,Join_Count,TARGET_FID,CASE_ID,COLLISION_,COLLISION1,DAY_OF_WEE,COLLISION2,PARTY_COUN,PCF_VIOL_C,...,AnalysisArea,SHAPE,index_right,FID,ZIP,COMMUNITY,SHAPE_STAr,SHAPE_STLe,Shape__Area,Shape__Length
0,1,3,12,6560089,2014-07-05,2035,6,3,2,08,...,0.003392,"{""x"": -117.16041008955163, ""y"": 32.74694011979...",82,83,92103,San Diego,1.012375e+08,57977.990851,1.333285e+07,21037.430975
1,2,3,13,6560089,2014-07-05,2035,6,3,2,08,...,0.003392,"{""x"": -117.16041008955163, ""y"": 32.74694011979...",82,83,92103,San Diego,1.012375e+08,57977.990851,1.333285e+07,21037.430975
2,3,4,14,6560090,2014-07-02,930,3,4,2,08,...,0.001458,"{""x"": -117.14752151011444, ""y"": 32.74118107036...",82,83,92103,San Diego,1.012375e+08,57977.990851,1.333285e+07,21037.430975
3,4,4,15,6560090,2014-07-02,930,3,4,2,08,...,0.001458,"{""x"": -117.14752151011444, ""y"": 32.74118107036...",82,83,92103,San Diego,1.012375e+08,57977.990851,1.333285e+07,21037.430975
4,31,1,59,6572811,2014-07-19,1350,6,4,2,09,...,0.008725,"{""x"": -117.16127197006615, ""y"": 32.74089859979...",82,83,92103,San Diego,1.012375e+08,57977.990851,1.333285e+07,21037.430975


Collisions in bike lanes: 868
Collisions outside of bike lanes: 1133


Upon initial glance at the data, it seems like you are not as likely to get into a bike related accident in a bike lane, thus making it safer to ride in the bike lane. 

In [234]:
print('The number of zipcodes that have collision points.')
print(len(all_collision_with_zip['ZIP'].unique())) #Lost zipcodes when we spatial joined by zipcode
print('The number of zipcodes that have collision points on a bike lane.')
print(len(intersecting_collision_with_zip['ZIP'].unique())) #Lost zipcodes when we spatial joined by zipcode

The number of zipcodes that have collision points.
42
The number of zipcodes that have collision points on a bike lane.
36


When we spatial joined the collisions coordinates with the county zip code coordinates, we lost some zipcodes in the process because some zipcodes don't have any collision points, and some zipcodes don't have collisions intersecting with a bike lanes.

__How would the results differ for injuries of different severity?__

In [235]:
#Analyzing accidents by different severities for all collisions and intersecting collisions
all_collisions_severity_counts = all_collision_with_zip.groupby('COLLISION_SEVERITY').count()[['CASE_ID']].rename(columns={"CASE_ID": "Total Number of Bike Accidents"})
intersecting_collisions_severity_counts = intersecting_collision_with_zip.groupby('COLLISION2').count()[['CASE_ID']].rename(columns={"CASE_ID": "Number of Bike Accidents In Bike Lanes"})

severity_collisions = pd.merge(all_collisions_severity_counts, intersecting_collisions_severity_counts, left_index=True, right_index=True)
severity_collisions["Rate of Accidents in Bike Lane"] = severity_collisions['Number of Bike Accidents In Bike Lanes']/severity_collisions['Total Number of Bike Accidents']
severity_collisions.sort_values("Rate of Accidents in Bike Lane", ascending = False)


,Total Number of Bike Accidents,Number of Bike Accidents In Bike Lanes,Rate of Accidents in Bike Lane
COLLISION_SEVERITY,,,
4,1210,538,0.444628
3,2168,932,0.429889
2,202,79,0.391089
1,14,4,0.285714


* 1 - Fatal
* 2 - Injury (Severe)
* 3 - Injury (Other Visible)
* 4 - Injury (Complaint of Pain)

__How would the results differ for injuries of different zip codes?__

In [236]:
# Analyzing accidents by different zip codes for all collisions and intersecting collisions
all_collisions_grouped = all_collision_with_zip.groupby('ZIP').count()[['CASE_ID']].rename(columns={"CASE_ID": "Total Number of Bike Accidents"})
intersecting_collisions_grouped = intersecting_collision_with_zip.groupby('ZIP').count()[['CASE_ID']].rename(columns={"CASE_ID": "Number of Bike Accidents In Bike Lanes"})


zip_code_collisions = pd.merge(all_collisions_grouped, intersecting_collisions_grouped, on = "ZIP")
zip_code_collisions["Rate of Accidents in Bike Lane"] = zip_code_collisions['Number of Bike Accidents In Bike Lanes']/zip_code_collisions['Total Number of Bike Accidents']
zip_code_collisions.sort_values("Rate of Accidents in Bike Lane", ascending = False)

,Total Number of Bike Accidents,Number of Bike Accidents In Bike Lanes,Rate of Accidents in Bike Lane
ZIP,,,
92182,6,6,1.000000
92067,1,1,1.000000
92173,14,12,0.857143
92014,22,16,0.727273
92101,422,272,0.644550
92116,77,47,0.610390
92103,144,86,0.597222
92108,64,38,0.593750
92102,135,80,0.592593


__How would the results differ for injuries by the type of other party in the accident?__

In [237]:
#Analyzing accidents by type of other party for all collisions and intersecting collisions

#Collisions grouped by the party type involved
all_collision_with_zip.groupby('PARTY_TYPE').count()
intersecting_collision_with_zip.groupby('PARTY_TYPE').count()

#Define other party as the party who was not at fault, then analyze the results of the at-fault party by their party type
at_fault_all = all_collision_with_zip[all_collision_with_zip['AT_FAULT'] == 'N'] #All Collision Parties who are at fault
at_fault_intersecting = intersecting_collision_with_zip[intersecting_collision_with_zip['AT_FAULT'] == 'N']#Intersecting Collision Parties who are at fault

count_all_at_fault_by_party = at_fault_all.groupby('PARTY_TYPE').count()[['CASE_ID']].rename(columns={"CASE_ID": "Total Number of All Bike Accidents"})
count_intersecting_at_fault_by_party = at_fault_intersecting.groupby('PARTY_TYPE').count()[['CASE_ID']].rename(columns={"CASE_ID": "Total Number of Bike Accidents In Bike Lanes"})

#zip_code_collisions["rate"] = zip_code_collisions['Number of Bike Accidents In Bike Lanes']/zip_code_collisions['TotalNumber of Bike Accidents']
merged_at_fault_party = pd.merge(count_all_at_fault_by_party, count_intersecting_at_fault_by_party, on = "PARTY_TYPE")
merged_at_fault_party["Rate of Accidents in Bike Lane"] = merged_at_fault_party['Total Number of Bike Accidents In Bike Lanes']/merged_at_fault_party['Total Number of All Bike Accidents']
merged_at_fault_party.sort_values("Rate of Accidents in Bike Lane", ascending = False)

,Total Number of All Bike Accidents,Total Number of Bike Accidents In Bike Lanes,Rate of Accidents in Bike Lane
PARTY_TYPE,,,
5,7,5,0.714286
2,27,15,0.555556
4,986,438,0.444219
1,737,302,0.409769
3,64,26,0.406250


* 1 - Driver (including Hit and Run)
* 2 - Pedestrian
* 3 - Parked Vehicle
* 4 - Bicyclist
* 5 - Other

This portion will help us do additional analysis with classification techniques. We'll be classifying collision severity based on the party sobriety, type of collision, and various location features (weather, zip code, and whether a collision was in a bike lane or not). 

In [238]:
def is_intersection(row):
    """This function checks whether a collision was in a bike lane or not"""
    if row in list(intersecting_collision_with_zip['CASE_ID'].unique()):
        return 'Y'
    else:
        return 'N'
    
all_collision_with_zip['INTERSECTING_W_BIKE_LANE'] = all_collision_with_zip['CASE_ID'].apply(is_intersection)
all_collision_with_zip.head()[['CASE_ID', 'INTERSECTING_W_BIKE_LANE']]

,CASE_ID,INTERSECTING_W_BIKE_LANE
0,6291500,Y
1,6291500,Y
2,6417345,N
3,7053051,N
4,7053051,N


In [239]:
# quick sanity check to make sure that the above collisions were classified correctly
display(intersecting_collision_with_zip[intersecting_collision_with_zip['CASE_ID'] == 6291500])
display(intersecting_collision_with_zip[intersecting_collision_with_zip['CASE_ID'] == 6417345])
display(intersecting_collision_with_zip[intersecting_collision_with_zip['CASE_ID'] == 7053051])

,OBJECTID,Join_Count,TARGET_FID,CASE_ID,COLLISION_,COLLISION1,DAY_OF_WEE,COLLISION2,PARTY_COUN,PCF_VIOL_C,...,AnalysisArea,SHAPE,index_right,FID,ZIP,COMMUNITY,SHAPE_STAr,SHAPE_STLe,Shape__Area,Shape__Length
1542,509,2,1212,6291500,2014-08-06,1624,3,3,2,22,...,0.001408,"{""x"": -117.103594240369, ""y"": 32.9119039400184...",108,109,92131,San Diego,3.911265e+08,137602.571318,5.171307e+07,50006.616039
1543,510,2,1213,6291500,2014-08-06,1624,3,3,2,22,...,0.001408,"{""x"": -117.103594240369, ""y"": 32.9119039400184...",108,109,92131,San Diego,3.911265e+08,137602.571318,5.171307e+07,50006.616039


,OBJECTID,Join_Count,TARGET_FID,CASE_ID,COLLISION_,COLLISION1,DAY_OF_WEE,COLLISION2,PARTY_COUN,PCF_VIOL_C,...,AnalysisArea,SHAPE,index_right,FID,ZIP,COMMUNITY,SHAPE_STAr,SHAPE_STLe,Shape__Area,Shape__Length


,OBJECTID,Join_Count,TARGET_FID,CASE_ID,COLLISION_,COLLISION1,DAY_OF_WEE,COLLISION2,PARTY_COUN,PCF_VIOL_C,...,AnalysisArea,SHAPE,index_right,FID,ZIP,COMMUNITY,SHAPE_STAr,SHAPE_STLe,Shape__Area,Shape__Length


In [240]:
X = all_collision_with_zip[['PARTY_SOBRIETY', 'TYPE_OF_COLLISION','WEATHER_1', 'ZIP', 'INTERSECTING_W_BIKE_LANE']]
y = all_collision_with_zip['COLLISION_SEVERITY']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [241]:
# test different classifiers 

from sklearn.tree import DecisionTreeClassifier 
print('Decision Tree Classifier')
DT_clf = DecisionTreeClassifier(max_depth = 2)
pl = Pipeline(steps=[
            ('onehot', OneHotEncoder(handle_unknown = 'ignore')),
            ('clf', DT_clf)
        ])

pl.fit(X_train, y_train)

print('Training Score: ' + str(pl.score(X_train,y_train)))
print('Testing Score: ' + str(pl.score(X_test, y_test)))

# ---------------------------

from sklearn.svm import SVC 
print('\nSupport Vector Classifier')
SVC_clf = SVC(kernel = 'linear')
pl = Pipeline(steps=[
            ('onehot', OneHotEncoder(handle_unknown = 'ignore')),
            ('clf', SVC_clf)
        ])

pl.fit(X_train, y_train)

print('Training Score: ' + str(pl.score(X_train,y_train)))
print('Testing Score: ' + str(pl.score(X_test, y_test)))

# ---------------------------

from sklearn.ensemble import RandomForestClassifier
print('\nRandom Forest Classifier')
RF_clf = RandomForestClassifier()
pl = Pipeline(steps=[
            ('onehot', OneHotEncoder(handle_unknown = 'ignore')),
            ('clf', RF_clf)
        ])

pl.fit(X_train, y_train)

print('Training Score: ' + str(pl.score(X_train,y_train)))
print('Testing Score: ' + str(pl.score(X_test, y_test)))

# ---------------------------

from sklearn.neighbors import KNeighborsClassifier 
print('\nK-Nearest Neighbors Classifier')
KNN_clf = KNeighborsClassifier(7)
pl = Pipeline(steps=[
            ('onehot', OneHotEncoder(handle_unknown = 'ignore')),
            ('clf', KNN_clf)
        ])

pl.fit(X_train, y_train)

print('Training Score: ' + str(pl.score(X_train,y_train)))
print('Testing Score: ' + str(pl.score(X_test, y_test)))

Decision Tree Classifier
Training Score: 0.6023856858846919
Testing Score: 0.6061167747914736

Support Vector Classifier
Training Score: 0.6083499005964215
Testing Score: 0.6098239110287303

Random Forest Classifier
Training Score: 0.778131212723658
Testing Score: 0.623725671918443

K-Nearest Neighbors Classifier
Training Score: 0.6743538767395626
Testing Score: 0.577386468952734


In [242]:
# USE RLF FEATURES DUE TO HIGHEST ACCURACY

features = {} 
for feature, importance in zip(X.columns, RF_clf.feature_importances_):
    features[feature] = importance 

importances = pd.DataFrame.from_dict(features, orient='index').rename(columns={0: 'Feature Importance'}).sort_values('Feature Importance', ascending=False)
importances

,Feature Importance
TYPE_OF_COLLISION,0.036897
PARTY_SOBRIETY,0.015871
INTERSECTING_W_BIKE_LANE,0.012066
ZIP,0.011023
WEATHER_1,0.010594


##### Question 2

__In which parts of the city are you more likely to get into an alcohol-related accident?__

In [252]:
# Number of alcohol involved accidents per zip code
num_of_alc_involved_all = all_collision_with_zip.groupby('ZIP').agg(sum)[['ALCOHOL_INVOLVED']].rename(columns={"ALCOHOL_INVOLVED": "Total Number of Bike Accidents Involving Alc"}) #Sum of All the alcohol involved accidents grouped by zipcodes
total_collisions = all_collision_with_zip.groupby('ZIP').count()[['CASE_ID']].rename(columns={"CASE_ID": "Total Number of Bike Accidents"})  #Sum of all alcohol involved bikelane accidents grouped by zipcodes

alc_involved_collisions = pd.merge(total_collisions, num_of_alc_involved_all, on="ZIP")
alc_involved_collisions["Rate of Alcohol Related Accident"] = alc_involved_collisions['Total Number of Bike Accidents Involving Alc']/alc_involved_collisions['Total Number of Bike Accidents']
print("DataFrame w/ areas in San Diego city with the highest rate of alcohol related collisions:")
alc_involved_collisions.sort_values(by = "Rate of Alcohol Related Accident", ascending=False).iloc[:10]

DataFrame w/ areas in San Diego city with the highest rate of alcohol related collisions:


,Total Number of Bike Accidents,Total Number of Bike Accidents Involving Alc,Rate of Alcohol Related Accident
ZIP,,,
92182,6,2,0.333333
92136,8,2,0.250000
92109,439,75,0.170843
92113,104,15,0.144231
92093,21,3,0.142857
92173,14,2,0.142857
92107,122,15,0.122951
92139,17,2,0.117647
92110,171,18,0.105263


__How can you explain that using additional statistics by zip codes, for example, by the number of alcohol-serving bars within the zip code?__

In [255]:
# Alcohol related accidents vs non-alcohol accidents
all_alcohol_related = all_collision_with_zip.groupby('ALCOHOL_INVOLVED').count()
intersecting_alcohol_related = intersecting_collision_with_zip.groupby('ALCOHOL_IN').count()

In [256]:
#Create simple  model to predict if alcohol was involved in the accident based on Geoenriched features
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(class_weight= 'balanced')

#using all_collisions dataframe
collisions_with_geoenriched = all_collision_with_zip.join(zip_collision_counts_df, on='ZIP', how = 'inner') #Joining with geoenriched dataframe
#collisions_with_geoenriched['COLLISION_DATE'] = pd.to_datetime(date_time)

cols = ['TYPE_OF_COLLISION', 'Disposable_Income', 'Number_of_Alcohol_Stores', 'Number_of_Bars'] #'PARTY_COUNT', 'Number_of_Bike_Collisions',


X = collisions_with_geoenriched[cols]
y = collisions_with_geoenriched['ALCOHOL_INVOLVED']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

DT_clf = DecisionTreeClassifier(class_weight = 'balanced')
RF_clf = RandomForestClassifier(class_weight = 'balanced'
                               )
pl = Pipeline(steps=[
            ('onehot', OneHotEncoder(handle_unknown = 'ignore')),
            ('clf', DT_clf)
        ])

pl.fit(X_train, y_train)

preds = pl.predict(X_test)
accuracy = pl.score(X_test,y_test)
print("Accuracy: " + str(accuracy))

TP = sum([(p and l) for (p,l) in zip(preds, y)])
FP = sum([(p and not l) for (p,l) in zip(preds, y)])
TN = sum([(not p and not l) for (p,l) in zip(preds, y)])
FN = sum([(not p and l) for (p,l) in zip(preds, y)])
TPR = TP/ (TP+FN)
TNR = TN/ (TN+FP)
print("True Positive Rate: " + str(TPR))
print("True Negative Rate: " + str(TNR))

features = {} 
for feature, importance in zip(X.columns, DT_clf.feature_importances_):
    features[feature] = importance 


importances = pd.DataFrame.from_dict(features, orient='index').rename(columns={0: 'Feature Importance'}).sort_values('Feature Importance', ascending=False)
print('Decision Tree Features')
display(importances)

pl = Pipeline(steps=[
            ('onehot', OneHotEncoder(handle_unknown = 'ignore')),
            ('clf', RF_clf)
        ])

pl.fit(X_train, y_train)

preds = pl.predict(X_test)
accuracy = pl.score(X_test,y_test)
print("Accuracy: " + str(accuracy))

TP = sum([(p and l) for (p,l) in zip(preds, y)])
FP = sum([(p and not l) for (p,l) in zip(preds, y)])
TN = sum([(not p and not l) for (p,l) in zip(preds, y)])
FN = sum([(not p and l) for (p,l) in zip(preds, y)])
TPR = TP/ (TP+FN)
TNR = TN/ (TN+FP)
print("True Positive Rate: " + str(TPR))
print("True Negative Rate: " + str(TNR))

features = {} 
for feature, importance in zip(X.columns, DT_clf.feature_importances_):
    features[feature] = importance 
importances = pd.DataFrame.from_dict(features, orient='index').rename(columns={0: 'Feature Importance'}).sort_values('Feature Importance', ascending=False)

print('Random Forest Features')
display(importances)

Accuracy: 0.604494382022472
True Positive Rate: 0.4875
True Negative Rate: 0.5790123456790124
Decision Tree Features


,Feature Importance
Number_of_Alcohol_Stores,0.080278
Disposable_Income,0.031231
TYPE_OF_COLLISION,0.025920
Number_of_Bars,0.025565


Accuracy: 0.6786516853932584
True Positive Rate: 0.3875
True Negative Rate: 0.6851851851851852
Random Forest Features


,Feature Importance
Number_of_Alcohol_Stores,0.080278
Disposable_Income,0.031231
TYPE_OF_COLLISION,0.025920
Number_of_Bars,0.025565


# Conclusion and Discussion<a class="anchor" id="conclusion-bullet"></a>

* __Question: Are you more or less likely to be injured in a bike-related accident if you are on a bike path or not? How would the results differ for injuries of different severity, zip codes, and by the type of other party in the accident?__

According to our dataset, you are less likely to be injured in a bike-related accident if you are on a bike path. In order to explain how we came to this conclusion, we'll explain the methodology behind our answer. We first started with a dataset that contained the collision points for all of San Diego city. We then joined the collision points data set with the data set that explained the different types of parties. The resulting dataset was one where each row was a party member of a collision. This meant that multiple rows could have the same case ID because each row equates to one party member. After creating point coordinates from the longitude and latitude columns, we did a join_features operation that helped us find the collisions that happened within 10 feet of a bike lane. We found that 868 collisions happened on a bike lane whereas 1133 collisions happened outside of bike lanes. It's pretty intuitive that bike lanes would be safer, considering that they are designated for bikers. 

Upon reviewing how the results differ for injuries of different severity, we noticed that most injuries are not severe. We took the rate of injuries rather than comparing raw numbers because we wanted to get a ratio of how many accidents happened in bike lanes compared to outside of bike lanes. We also didn't want to take raw numbers because a larger number of overall collisions will obviously lead to a larger number of collisions in bike lanes. Our data shows that most injuries were either a complaint of pain (rate: 44.4628%) or a visible injury (rate: 42.9889%) for collisions in bike lanes out of all collisions. This indicates, that collisions in bike lanes are not too fatal. We also reviewed Rate of Accidents in Bike Lane by zip codes. The top 5 zip codes with the highest rate of bike collisions in the bike lanes out of all collsions are: 92182, 92067, 92173, 92014, and 92101. Lastly, we also looked at the rate of collisions in bike lanes out of all collisions for the other party involved. We describe the other party as the people who were NOT at fault. We found that most collision in bike lanes happened to hit the category "Other" (rate: 71.4286%) and pedestrians (rate: 55.5556%). 

Lastly, we performed classification in order to see if we could classify collision severity based on the party sobriety, type of collision, and various location features (weather, zip code, and whether a collision was in a bike lane or not). We wanted to do this in order to see if whether a collision was in a bike lane or not played a big role in how severe a collision is. This would help us decipher whether being in a bike lane or not could play a roll in collision severity. Since our Random Forest Classifier performed the best on the test (score: ~0.778) and training set  (score: ~0.624) we decided to see which factors played the most important role in the classification. We saw that the columns ranged in importance in this order: type of collision, party sobriety, whether a collision was intersecting with a bike lane, zip code, and lastly the weather. So although the collision being in a bike lane was not the lowest in importance, the more important columns were the type of collision and the party sobriety. This is intuitive considering a collision could be more severe when you get information on what kind of collision it is and whether or not anyone was intoxicated. 

* __Question: In which parts of the city are you more likely to get into an alcohol-related accident? How can you explain that using additional statistics by zip codes, for example, by the number of alcohol-serving bars within the zip code?__

To answer the question of what parts of the city are you most likely to get into an alcohol-related accident we first gathered the datasets that were needed. To do so we got a feature layer of San Diego zip codes polygon layer and spatial joined it with our Collisions data, and we made sure that we are only using the zip codes that are included in San Diego City. Then we performed exploratory data analysis on the joined collision and zipcodes datasets and filtered the rows to only include the collisions that occurred with alcohol being involved (we filtered using the ‘ALCOHOL_INVOLVED’ column of the ‘Collisions.csv’ dataset). We then proceeded to analyze the results by grouping the filtered dataset by the ‘ZIP’ column and aggregated using a count function to calculate the alcohol-related accidents for each zip code. To analyze our aggregated results we calculated the rate of bike-related accidents that are alcohol-involved (# of alcohol-related accidents / # of total accidents), then sorted the values and visualized the highest rate zipcodes. In our results, the top 3 zipcodes with the highest alcohol-related accident was [92182, 92136, 92109]  with respective rates of [33%, 25%, 17%] followed by other zipcodes that are in the ~0.15 range.

To get more statistics by zip codes to answer the next question we performed some geonriching. During the geoenriching process, we found that the "Disposable_Income_Profile_rep" and the "retailmarketplace" data collections helped us find relevant features for future regression. We looked at datasets also given in the instructions (EX: businesses, GroceryAlcoholicBeverages, LeisureActivitiesLifestyle, food), but we found that these were a little more along the line of what we wanted to use as features. The columns that we extracted are explained in the comments in the cell above this. We chose to see the disposable income of a zip code in order to see if maybe higher disposable incomes lead to a higher number of alcohol stores and bars for people to spend their money at. We also extracted the number of businesses of Beer/Wine/Liquor Stores and Alcohol Drinking Places (aka bars) via NAICS codes. We did this so that we could see if there were a higher number of alcohol-related bike crashes as a result of a higher number of both of those kinds of stores.

To further analyze our results we created a machine learning classifier analysis model on the geonriched features of ‘Number of Alcohol Serving Places’, ‘Disposable Income’, and ‘Number of Bars’ to predict whether an accident was involved with alcohol or not. With an accuracy score of ~70% we found that the most highly correlated features were the ‘Number of Alcohol Serving Places’. The results infer that in zip code areas of the city that have a high number of alcohol-serving places, alcohol-related accidents are more likely to happen. 

